In [13]:
import os
import time
import pandas as pd
from Declare4Py.ProcessMiningTasks.ASPLogGeneration.ASPUtils.damerauLevenshtein import DamerauLevenshteinDistance
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPLogGenerator import AspGenerator

Test incrociato per cercare la migliore combinazione tra configurazione, frequenza e threads per migliorare la velocita' del solver

In [14]:
model_name = 'data-model1'
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))

# Number of cases that have be generated
num_of_cases = 20

# Minimum and maximum number of events a case can contain
(num_min_events, num_max_events) = (5,15)

asp_gen: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events, verbose=True)

DEBUG:ASP generator:Distribution for traces UNIFORM
DEBUG:ASP generator:traces: 20, events can have a trace min(5) max(15)


In [9]:
%%time
# "frumpy",
configuration = ["frumpy","tweety", "crafty", "jumpy", "trendy", "handy"]
frequency = [0, 0.3, 0.6, 0.9, 1]
sign_def = ["rnd", "asp"]
modes = ["optN", "ignore"]
strategy = [None]
heuristic = [None]

for config in configuration:
    for freq in frequency:
        for sign in sign_def:
            for mode in modes:
                for strat in strategy:
                    for heu in heuristic:
                
                        asp_gen.use_custom_clingo_configuration(config, None, freq, sign, mode, strat, heu)
                        start = time.time()
                        
                        asp_gen.run()
                        
                        end = time.time()
                        asp_gen.to_csv(f'Log_Results/time_{round(end - start, 4)}_config_{config}_threads_{os.cpu_count()}_freq_{freq}_sign_{sign}_mode_{mode}_strategy_{strat}_heuristic_{heu}_.csv') 



DEBUG:Distribution:Distribution() UNIFORM min_mu: 5 max_sigma: 15 num_traces: 20 custom_prob: None
DEBUG:Distribution:Calculating Uniform Distribution
DEBUG:Distribution:Generating Uniform Probabilities since either distribution is uniform or custom probabilities are None
DEBUG:Distribution:Probabilities sum is 1
DEBUG:Distribution:Distribution result: [ 9  7  8 12 12  9 14  7 13 10 10  6  8  9  7 11  9  5 13 10]
DEBUG:ASP generator:Prepared distribution of 20 positive traces with distribution: Counter({9: 4, 7: 3, 10: 3, 8: 2, 12: 2, 13: 2, 14: 1, 6: 1, 11: 1, 5: 1})
DEBUG:ASP generator:Prepared distribution of 0 negative traces with distribution: Counter()
DEBUG:ASP generator:Generating positive Traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Total trace to generate and events: Traces:4, Events: 9, Seed:466018868
DEBUG:ASP generator: Traces gene

CPU times: total: 1h 33min 41s
Wall time: 19min 42s


Filtraggio dei risultati per thread creando dei report file per thread

In [18]:
%%time

Cols_Name = ["Configuration", "Distance", "Time", "Frequency", "Sign-Def", "Opt-Mode", "Strategy", "Heuristic"]
results = []

t = 16

for file in os.listdir("Log_results"):
    
    stripped_file = file.split("_")
    
    i = 0
    if stripped_file[0] == "distance":
        if stripped_file[1] == "report":
            continue
        else:
            i = 2
    
    print(stripped_file)
    
    time = stripped_file[1 + i]
    config = stripped_file[3 + i]
    # thread = stripped_file[5 + i]
    freq = stripped_file[7 + i]
    sign = stripped_file[9 + i]
    mode = stripped_file[11 + i]
    strat = stripped_file[13 + i]
    heu = stripped_file[15 + i]
    
    if i == 0:
        distance = DamerauLevenshteinDistance.analize_csv(f"Log_results/{file}")
    else:
        distance = stripped_file[1]
        
    results.append([config, str(round(float(distance),4)), time, freq, sign, mode, strat, heu])
    
    if i == 0:
        try:
            os.rename("Log_results/" + file, f"Log_results/distance_{round(distance, 4)}_" + file)
        except FileNotFoundError:
            print("file not found")
            
if len(results) > 0:
    df = pd.DataFrame(results, columns=Cols_Name)
    df.to_csv(f"Log_results/distance_report_threads_{t}.csv")

['distance', '10.0579', 'time', '12.674', 'config', 'trendy', 'threads', '16', 'freq', '0.3', 'sign', 'asp', 'mode', 'optN', 'strategy', 'None', 'heuristic', 'None', '.csv']
['distance', '10.2842', 'time', '13.6197', 'config', 'tweety', 'threads', '16', 'freq', '1', 'sign', 'asp', 'mode', 'optN', 'strategy', 'None', 'heuristic', 'None', '.csv']
['distance', '10.4474', 'time', '20.4473', 'config', 'frumpy', 'threads', '16', 'freq', '1', 'sign', 'asp', 'mode', 'ignore', 'strategy', 'None', 'heuristic', 'None', '.csv']
['distance', '2.8895', 'time', '5.124', 'config', 'frumpy', 'threads', '16', 'freq', '0', 'sign', 'rnd', 'mode', 'optN', 'strategy', 'None', 'heuristic', 'None', '.csv']
['distance', '3.7632', 'time', '5.1026', 'config', 'frumpy', 'threads', '16', 'freq', '0', 'sign', 'rnd', 'mode', 'ignore', 'strategy', 'None', 'heuristic', 'None', '.csv']
['distance', '3.7684', 'time', '5.2865', 'config', 'frumpy', 'threads', '16', 'freq', '0', 'sign', 'asp', 'mode', 'optN', 'strategy', '

test configurations

In [15]:
asp_gen.use_custom_clingo_configuration("trendy", None, 0.3, "asp", "optN", None, None)

for i in range(5):
    start = time.time()
    
    asp_gen.run()
    
    end = time.time()
    
    file = f"time_{round(end - start, 4)}_config_trendy_threads_{os.cpu_count()}_freq_0.3_sign_asp_mode_optN_strategy_{None}_heuristic_{None}_.csv"
    asp_gen.to_csv(f"Log_Results/{file}") 
    distance = DamerauLevenshteinDistance.analize_csv(f"Log_results/{file}")
    os.rename("Log_results/" + file, f"Log_results/distance_{round(distance, 4)}_" + file)

DEBUG:Distribution:Distribution() UNIFORM min_mu: 5 max_sigma: 15 num_traces: 20 custom_prob: None
DEBUG:Distribution:Calculating Uniform Distribution
DEBUG:Distribution:Generating Uniform Probabilities since either distribution is uniform or custom probabilities are None
DEBUG:Distribution:Probabilities sum is 1
DEBUG:Distribution:Distribution result: [ 9  8  8  8 10  6 11  5 15 13  6 11 10 13 11 13  7 12 12 12]
DEBUG:ASP generator:Prepared distribution of 20 positive traces with distribution: Counter({8: 3, 11: 3, 13: 3, 12: 3, 10: 2, 6: 2, 9: 1, 5: 1, 15: 1, 7: 1})
DEBUG:ASP generator:Prepared distribution of 0 negative traces with distribution: Counter()
DEBUG:ASP generator:Generating positive Traces
DEBUG:ASP generator:Translate declare model to ASP
DEBUG:ASP generator:Declare model translated to ASP. Total Facts 13
DEBUG:ASP generator:ASP encoding generated
DEBUG:ASP generator:Total trace to generate and events: Traces:1, Events: 9, Seed:283674927
DEBUG:ASP generator: Traces gene

Analisi di due altri modelli declare che dovrebbero causare problemi al solver. Si cerca correttezza nei modelli

In [ ]:
import os
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPLogGenerator import AspGenerator
from Declare4Py.ProcessMiningTasks.ASPLogGeneration.ASPTranslator.asp_translator import ASPModel

In [ ]:
model1_name = "data-model1"
model2_name = "data-model2"

model1: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model1_name}.decl"))
model2: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model2_name}.decl"))

# Number of cases that have be generated
num_of_cases = 10

# Minimum and maximum number of events a case can contain
(num_min_events, num_max_events) = (10,20)

# activity, attribute, events encoding
encode = True

model1_gen: AspGenerator = AspGenerator(model1, num_of_cases, num_min_events, num_max_events, encode)
model2_gen: AspGenerator = AspGenerator(model2, num_of_cases, num_min_events, num_max_events, encode)

In [ ]:
print(ASPModel(False).from_decl_model(model1_gen.process_model, None))

In [ ]:
%%time

model1_gen.run()
model1_gen.to_csv(f"Log_results/{model1_name}.csv")

In [ ]:
print(model1_gen.lp_model)

In [ ]:
%%time

model2_gen.run()
model2_gen.to_csv(f"Log_results/{model2_name}.csv")

Controlliamo quali solver rispettano le regole asp

In [ ]:
import os
import time
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPLogGenerator import AspGenerator

model_name = "data-model1-simplified"
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))


num_of_cases = 1

# Minimum and maximum number of events a case can contain
(num_min_events, num_max_events) = (20, 20)

# activity, attribute, events encoding
encode = True

asp: AspGenerator = AspGenerator(model, num_of_cases, num_min_events, num_max_events, encode, verbose=False)

In [ ]:

start = time.time()
asp.run()
end = time.time()

asp.to_csv(f'Log_Results/default.csv') 

In [ ]:
asp.to_xes(f'Log_Results/default.xes')

fix float numbers

In [ ]:
%%time
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.LogGenerator.ASP.ASPLogGenerator import AspGenerator
import os

model_name = "float_test_model"
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))

asp: AspGenerator = AspGenerator(model, 10, 5, 10, verbose=True)
asp.run()
asp.to_csv(f'Test/float_test.csv')

In [ ]:
from Declare4Py.ProcessModels.DeclareModel import DeclareModel
from Declare4Py.ProcessMiningTasks.ASPLogGeneration.asp_generator import AspGenerator as OldASPGenerator
import os

model_name = "data-model1-simplified"
model: DeclareModel = DeclareModel().parse_from_file(os.path.join("../../../", "tests", "test_models", f"{model_name}.decl"))

asp: OldASPGenerator = OldASPGenerator(model, 30, 10, 10)
asp.run()
asp.to_csv(f'Config_Test/manpreet.csv')


In [ ]:
sorted_list = sorted(similarity_dict.items())
filtered_list = []

threshold = 45

for csv_file in sorted_list:
    if csv_file[1] > threshold:
        filtered_list.append(csv_file)
        
filtered_list.sort(key = lambda x: x[1], reverse = True)

files = [file[0] for file in filtered_list]

def get_similarity(csv_file: str):
    for name, similarity in filtered_list:
        if name == csv_file:
            return similarity
    return 0

for file in os.listdir("Log_results"):
    if file not in files:
        try:
            os.remove(f"Log_results/{file}")
        except FileNotFoundError:
            pass
    else:
        try:
            os.rename("Log_results/" + file, f"Log_results/similarity_{get_similarity(file)}_" + file)
        except FileNotFoundError:
            print("file not found")
            pass
            

In [ ]:
sorted_list = sorted(similarity_dict.items())

def get_similarity(csv_file: str):
    for name, similarity in filtered_list:
        if name == csv_file:
            return similarity
    return 0

for file in os.listdir("Log_results"):
    try:
        os.rename("Log_results/" + file, f"Log_results/similarity_{get_similarity(file)}_" + file)
    except FileNotFoundError:
        print("file not found")
        pass
